In [1]:
%load_ext autoreload
%autoreload 2
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime

from typing import Callable, List

from psycopg import sql

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch
from dmp.postgres_interface.element.column import Column
from dmp.postgres_interface.element.table import Table
from dmp.postgres_interface.element.column_group import ColumnGroup

from dmp.util.butter_e_export import *

pd.options.display.max_seq_items = None
credentials = jobqueue.load_credentials("dmp")

2023-11-15 15:26:06.308339: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 15:26:06.328850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 15:26:06.328869: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 15:26:06.328882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 15:26:06.333002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 15:26:06.333687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# epoch that minimizes: Column = Column("minimizes", "loss") and the corresponding loss
#

{
  "fit": {
    "epochs": 3000,
    "batch_size": 256
  },
  "loss": null,
  "seed": 1663277341,
  "type": "TrainingExperiment",
  "batch": "energy_1",
  "model": {
    "size": 2048,
    "type": "DenseBySize",
    "depth": 4,
    "inner": {
      "type": "Dense",
      "units": -1,
      "use_bias": true,
      "activation": "relu",
      "bias_constraint": null,
      "bias_initializer": "Zeros",
      "bias_regularizer": null,
      "kernel_constraint": null,
      "kernel_initializer": "GlorotUniform",
      "kernel_regularizer": null,
      "activity_regularizer": null
    },
    "input": null,
    "shape": "exponential",
    "output": null,
    "search_method": "integer"
  },
  "record": {
    "type": "ExperimentRecordSettings",
    "model": null,
    "times": false,
    "metrics": null,
    "post_training_metrics": false
  },
  "dataset": {
    "name": "201_pol",
    "type": "DatasetSpec",
    "method": "shuffled_train_test_split",
    "source": "pmlb",
    "test_split": 0.2,
    "label_noise": 0,
    "validation_split": 0
  },
  "run_tags": null,
  "optimizer": {
    "class": "Adam",
    "learning_rate": 0.0001
  },
  "precision": "float32",
  "early_stopping": null
}

In [7]:
from psycopg import ClientCursor


print(f"run vars {vars(run)}")

columns = (
    run
    + ColumnGroup(*[c for c in job_status.columns if c.name != "id"])
    + job_data.command
)
print(columns.names)


def passthrough(row, index, value, column, data):
    data[column.name] = value


column_converters: List[Callable] = [passthrough for _ in columns]


def flatten_json(json_obj, destination=None, parent_key="", separator="_"):
    if isinstance(destination, dict):
        flattened = destination
    else:
        flattened = {}

    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


column_converters[
    columns.get_index_of(job_data.command)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)
column_converters[
    columns.get_index_of(run.run_data)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)


def parquet_to_dataframe(row, index, value, column, data):
    with io.BytesIO(value) as buffer:
        data[column.name] = (
            pyarrow.parquet.read_table(pyarrow.PythonFile(buffer, mode="r"))
            .to_pandas()
            .sort_values(by="epoch")
        )


column_converters[columns.get_index_of(run.run_history)] = parquet_to_dataframe
column_converters[columns.get_index_of(run.run_extended_history)] = parquet_to_dataframe


dfs = []

with ConnectionManager(credentials) as connection:
    """
    TODO: convert this into two steps:
    1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

          SELECT
                  {run}.{run_id}
          FROM
                  {run},
                  {job_status},
                  {job_data}
          WHERE TRUE
                  AND {run}.batch like {pattern}
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
              AND {job_status}.status = 2
          ORDER BY experiment_id, run_id;

    2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
      -> partition by the attributes we care about querying by (dataset, size, shape, depth)

              SELECT
                  {columns}
              FROM
                  {run},
                  {job_status},
                  {job_data}
              WHERE TRUE
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
                  AND {job_status}.status = 2
                  AND {run}.{run_id} IN ({ids})

        pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
        results = pool.uimap(download_chunk, chunks)
        for num_rows, chunk in results:
            num_stored += 1
            print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

    extra credit) extract butter data matching this as well into a new dataset
    extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels

    getting a few attrs from the energy runs:
    SELECT
        size,
        depth,
        shape,
        dataset,
        learning_rate,
        batch_size,
        optimizer,
        COUNT(1) num,
        SUM(is_cpu) num_cpu,
        (COUNT(1) - SUM(is_cpu)) num_gpu
    FROM
    (
            SELECT
                    slurm_job_id,
                    start_time,
                    update_time end_time,
                    (command->'model'->'size')::int size,
                    (command->'model'->'depth')::int depth,
                    (command->'model'->>'shape')::text shape,
                    (command->'dataset'->>'name')::text dataset,
                    (command->'optimizer'->'learning_rate')::float learning_rate,
                    (command->'fit'->'batch_size')::int batch_size,
                    (command->'optimizer'->>'class')::text optimizer,
                    (num_gpus <= 0)::int is_cpu
            FROM
                    run,
                    job_status,
                    job_data
            WHERE TRUE
                    AND run.batch like '%energy%'
                    AND job_status.id = run.run_id
                    AND job_status.id = job_data.id
                    AND job_status.status = 2
    ) x
    WHERE TRUE
            AND depth = 5
            AND shape = 'rectangle'
            AND dataset = 'sleep'
    GROUP BY
            learning_rate,
            batch_size,
            optimizer,
            depth,
            shape,
            dataset,
            size
    ORDER BY
            learning_rate,
            batch_size,
            optimizer,
            depth,
            shape,
            dataset,
            size
    ;



    """
    query = psycopg.sql.SQL(
        """
SELECT
	{columns}
FROM
	{run},
	{job_status},
	{job_data}
WHERE TRUE
	AND {run}.batch like {pattern}
	AND {job_status}.id = {run}.run_id
	AND {job_status}.id = {job_data}.id
    AND {job_status}.status = 2
ORDER BY experiment_id, run_id
LIMIT 10;
"""
    ).format(
        columns=columns.columns_sql,
        run=run.identifier,
        job_status=job_status.identifier,
        job_data=job_data.identifier,
        pattern=sql.Literal("%energy%"),
    )

    with ClientCursor(connection) as c:
        print(c.mogrify(query))

    with connection.cursor(binary=True) as cursor:
        cursor.execute(query, binary=True)

        for row in cursor:
            row_data = {}
            for i, (column, column_converter) in enumerate(
                zip(columns, column_converters)
            ):
                column_converter(row, i, row[i], column, row_data)

            row_df = row_data["run_history"]
            row_df = row_df.join(
                row_data["run_extended_history"], on="epoch", how="left", rsuffix="_"
            )
            for k in ("run_history", "run_extended_history"):
                del row_data[k]

            for k, v in row_data.items():
                if k in row_df:
                    pass
                if isinstance(v, list):
                    row_df[k] = [v] * len(row_df)
                else:
                    row_df[k] = v
                # data.setdefault(k, []).append(v)
            dfs.append(row_df)

data = pandas.concat(dfs)
del dfs

data.describe()
print(data.head())

run vars {'_name': 'run', '_columns': (), '_index': None}
('experiment_id', 'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id', 'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory', 'host_name', 'batch', 'run_data', 'run_history', 'run_extended_history', 'queue', 'status', 'priority', 'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent', 'command')
{Column(_name='experiment_id', type_name='uuid'): 0, Column(_name='run_timestamp', type_name='timestamp'): 1, Column(_name='run_id', type_name='uuid'): 2, Column(_name='job_id', type_name='uuid'): 3, Column(_name='seed', type_name='bigint'): 4, Column(_name='slurm_job_id', type_name='bigint'): 5, Column(_name='task_version', type_name='smallint'): 6, Column(_name='num_nodes', type_name='smallint'): 7, Column(_name='num_cpus', type_name='smallint'): 8, Column(_name='num_gpus', type_name='smallint'): 9, Column(_name='gpu_memory', type_name='integer'): 10, Column(_name='host_name', type_name='text'): 11,

/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1595: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
data.columns

Index(['test_mean_squared_error', 'train_mean_squared_error', 'test_loss',
       'train_loss', 'epoch', 'test_cosine_similarity',
       'train_cosine_similarity', 'test_kullback_leibler_divergence',
       'train_kullback_leibler_divergence', 'test_root_mean_squared_error',
       'train_root_mean_squared_error', 'test_mean_absolute_error',
       'train_mean_absolute_error', 'test_mean_squared_logarithmic_error',
       'train_mean_squared_logarithmic_error', 'epoch_', 'experiment_id',
       'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id',
       'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory',
       'host_name', 'batch', 'cpus', 'gpus', 'nodes', 'git_hash', 'platform',
       'strategy', 'precision', 'record_model', 'record_times',
       'python_version', 'record_metrics', 'tensorflow_version',
       'record_post_training_metrics', 'queue', 'status', 'priority',
       'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent',
       

In [5]:
data

/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1595: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1595: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals 

,test_mean_squared_error,train_mean_squared_error,test_loss,train_loss,epoch,test_cosine_similarity,train_cosine_similarity,test_kullback_leibler_divergence,train_kullback_leibler_divergence,test_root_mean_squared_error,...,run_tags,class,learning_rate,early_stopping,test_accuracy,train_accuracy,test_hinge,train_hinge,test_squared_hinge,train_squared_hinge
0,0.018292,0.018138,0.018292,1.813805e-02,1,1.000000,0.999675,0.018892,1.879420e-02,0.131305,...,None,Adam,0.0001,None,NaN,NaN,NaN,NaN,NaN,NaN
1,0.017241,0.016899,0.017241,1.689905e-02,2,1.000000,0.999675,0.017697,1.692268e-02,0.127949,...,None,Adam,0.0001,None,NaN,NaN,NaN,NaN,NaN,NaN
2,0.016371,0.015950,0.016371,1.594961e-02,3,1.000000,0.999675,0.016885,1.618220e-02,0.125047,...,None,Adam,0.0001,None,NaN,NaN,NaN,NaN,NaN,NaN
3,0.015637,0.015154,0.015637,1.515436e-02,4,1.000000,0.999675,0.016182,1.565967e-02,0.122488,...,None,Adam,0.0001,None,NaN,NaN,NaN,NaN,NaN,NaN
4,0.015003,0.014478,0.015003,1.447835e-02,5,1.000000,0.999675,0.015552,1.521902e-02,0.120194,...,None,Adam,0.0001,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,NaN,NaN,0.437851,2.128737e-11,2996,0.984514,1.000000,0.209813,2.122752e-11,NaN,...,None,Adam,0.0001,None,0.984429,1.0,NaN,NaN,NaN,NaN
2996,NaN,NaN,0.437654,2.128737e-11,2997,0.984507,1.000000,0.209817,2.117616e-11,NaN,...,None,Adam,0.0001,None,0.984429,1.0,NaN,NaN,NaN,NaN
2997,NaN,NaN,0.437477,2.128737e-11,2998,0.984497,1.000000,0.209831,2.113641e-11,NaN,...,None,Adam,0.0001,None,0.984286,1.0,NaN,NaN,NaN,NaN
2998,NaN,NaN,0.437325,2.128737e-11,2999,0.984484,1.000000,0.209855,2.110676e-11,NaN,...,None,Adam,0.0001,None,0.984286,1.0,NaN,NaN,NaN,NaN
